In [32]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn

from PIL import Image

In [33]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()

        # Convolutional layer 1: 3 input channels, 16 output channels, 3x3 kernel size, stride of 1, padding of 1
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling with 2x2 kernel and stride of 2

        # Convolutional layer 2: 16 input channels, 32 output channels, 3x3 kernel size, stride of 1, padding of 1
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling with 2x2 kernel and stride of 2

        # Fully connected layer 1: Input size 32*56*56 (224/2^2), output size 64
        self.fc1 = nn.Linear(16 * 56 * 56, 64)
        self.relu3 = nn.ReLU()

        # Fully connected layer 2: Input size 64, output size 2
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.conv1(x)  # Apply convolutional layer 1
        x = self.relu1(x)  # Apply ReLU activation function
        x = self.pool1(x)  # Apply max pooling

        x = self.conv2(x)  # Apply convolutional layer 2
        x = self.relu2(x)  # Apply ReLU activation function
        x = self.pool2(x)  # Apply max pooling

        x = x.view(x.size(0), -1)  # Flatten the tensor

        x = self.fc1(x)  # Apply fully connected layer 1
        x = self.relu3(x)  # Apply ReLU activation function
        x = self.fc2(x)  # Apply fully connected layer 2

        return x

In [34]:
def predict(image):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to a consistent size
        transforms.ToTensor(),  # Convert images to tensors
        transforms.Normalize((0.5,), (0.5,))
        # transforms.Lambda(lambda x:x.view(-1))  #this flatten 28*28 into a 784 vector for each image
    ])
    
    # Load the saved model from file
    model = CNN(5)  # Instantiate your model
    model.load_state_dict(torch.load('./models/model_v1.pth'))

    model.eval()

    # Apply the transformation to the image
    image = transform(image).unsqueeze(0)
    
    # Make the prediction
    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_prob, predicted_class = torch.max(probabilities, 1)
    
    # # Return the predicted class and probability score
    # if predicted_class.item() == 0:
    #     authenticity = "Authentic"
    # else:
    #     authenticity = "Counterfeit"

    return predicted_class.item(), predicted_prob.item()

In [35]:
img = Image.open("test4.jpeg")
result = predict(img)

In [36]:
result

(3, 1.0)